In [3]:
import argparse
import os
import pandas as pd

dirs_ = os.listdir('./logs')
dirs_ = ['./logs/'+ dir_ for dir_ in dirs_]

final_dic = {'model': [],'config': [], 'score': [], 'monotone': [], 'screenshot': [], 'unknown': [], 'normal':[]}
configs = []
scores = []
for dir_ in dirs_:
#     print(dir_, '\n\n==============================\n')
    with open(dir_, 'r', encoding="utf8") as f :
        content = f.readlines()
        
        config_enter = 0
        config = ""
        
        score_count = 0
        scores = []
        monotones = []
        screenshots = []
        unknowns = []
        normals= [] 
        for i, line in enumerate(content):
            # line = line.replace('\n', '')
            if 'Loaded!' in line:
                final_dic['model'].append(line.split(' ')[0])

            if 'batch_size' in line:
                config_enter = 1
                config += line
            
            elif 'weight_decay' in line:
                config_enter = 0
                config += line
                
                final_dic['config'].append(config)
                config = ""
            
            elif config_enter:
                config += line
            
            if 'Final' in line:
                line = line.split(' ')
                scores.append(float(line[3]))
                normals.append(float(line[5].replace('),', '')))
                monotones.append(float(line[7].replace('),', '')))
                screenshots.append(float(line[9].replace('),', '')))
                unknowns.append(float(line[-1].replace(')]\n', '')))

                
            if 'train finished' in line:
                final_dic['score'].append(scores)
                final_dic['normal'].append(normals)
                final_dic['monotone'].append(monotones)
                final_dic['screenshot'].append(screenshots)
                final_dic['unknown'].append(unknowns)

                scores = [] ; monotones = [] ; screenshots = [] ; unknowns = [] ; normals= []

print([len(final_dic[a]) for a in final_dic])
if len(final_dic['config']) > len(final_dic['score']):
    final_dic['config'] = final_dic['config'][:-len(dirs_)]
    final_dic['model'] = final_dic['model'][:-len(dirs_)]
    
    
df = pd.DataFrame.from_dict(final_dic)
df['best_score'] = df['score'].apply(lambda x : max(x))
df['best_idx'] = df[['score', 'best_score']].apply(lambda x: x[0].index(x[1]), axis=1)
df['best_monotone'] = df[['monotone', 'best_idx']].apply(lambda x : x[0][x[1]], axis=1)
df['best_screenshot'] = df[['screenshot', 'best_idx']].apply(lambda x : x[0][x[1]], axis=1)
df['best_unknown'] = df[['unknown', 'best_idx']].apply(lambda x : x[0][x[1]], axis=1)
df['best_normal'] = df[['normal', 'best_idx']].apply(lambda x : x[0][x[1]], axis=1)

df = df.sort_values(by=['best_score'])
print("Top 15 models")
df[['model','best_score','best_idx','best_monotone', 'best_screenshot', 'best_unknown','best_normal']].iloc[-15:].iloc[::-1]


[37, 37, 37, 37, 37, 37, 37]
Top 15 models


,model,best_score,best_idx,best_monotone,best_screenshot,best_unknown,best_normal
18,VGG16,0.935651,10,0.974843,0.954635,0.917411,0.917799
1,ResNet50,0.932216,0,0.975155,0.939163,0.914676,0.922879
16,VGG16,0.930091,1,0.974843,0.938124,0.910745,0.920455
29,DenseNet121,0.929124,0,0.965517,0.957031,0.907110,0.911153
22,EfficientNet,0.928373,1,0.978056,0.946154,0.904237,0.910000
32,DenseNet121,0.928054,2,0.971787,0.954813,0.902369,0.908297
4,ResNet50,0.926433,0,0.962264,0.949416,0.904872,0.914072
6,ResNet50,0.924760,0,0.969512,0.935922,0.903301,0.915829
27,EfficientNet,0.924721,2,0.962733,0.950495,0.902439,0.908865
35,DenseNet121,0.922704,0,0.965732,0.943249,0.900229,0.907945


In [13]:
list_ = df[df['model']=='VGG16'][['config','best_score','score']].iloc[-5:].values.tolist()
list_.reverse()
for l in list_:
    print("Score = %1.5f \n"% l[1], l[2],'\n',l[0])

Score = 0.93565 
 [0.9148700491971562, 0.9156043576497404, 0.908508016981086, 0.9080030690441726, 0.924450238182049, 0.9157950309702374, 0.9259149456549863, 0.9136076727023984, 0.9243193476221724, 0.9264435058549543, 0.9356505782012194, 0.9181861290369105, 0.9220141944699981, 0.9159978047674434] 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [5, 10],
 'gamma': 0.3,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1.0000, 1.0000, 1.0000, 1.1000]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0.001}

Score = 0.93009 
 [0.9237617344202148, 0.9300912556874015, 0.9030766295670521, 0.9206538001121292, 0.9223542696401134] 
 {'batch_size': 128,
 'class_ratio': [0.5,

In [14]:
list_ = df[df['model']=='DenseNet121'][['config','best_score','score']].iloc[-5:].values.tolist()
list_.reverse()
for l in list_:
    print("Score = %1.5f \n"% l[1], l[2],'\n',l[0])

Score = 0.92912 
 [0.9291244063922011, 0.9138597020612601, 0.9263177597086122, 0.9196916646326336] 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [5, 10],
 'gamma': 0.3,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [1, None],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0}

Score = 0.92805 
 [0.9201343302982908, 0.9222771713432103, 0.9280536541578168, 0.9233389777622351, 0.9185997451445582, 0.9192908867795354] 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [0, 15],
 'gamma': 0.1,
 'learning_rate': 0.0001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)],
 'val_rat

In [15]:
list_ = df[df['model']=='ResNet50'][['config','best_score','score']].iloc[-5:].values.tolist()
list_.reverse()
for l in list_:
    print("Score = %1.5f \n"% l[1], l[2],'\n',l[0])

Score = 0.93222 
 [0.9322157906705133, 0.9155958263543363, 0.9259668308974776, 0.9125446608492649] 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [5, 10],
 'gamma': 0.3,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0.001}

Score = 0.92643 
 [0.9264334658419364, 0.913319076644429, 0.9242448133037332, 0.9250857479737512] 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [0, 15],
 'gamma': 0.1,
 'learning_rate': 0.0001,
 'loss_cls_weight': tensor([1.0000, 1.0000, 1.0000, 1.1000]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [2,
                Compose(
    ColorJit

In [7]:
list_ = df[df['model']=='EfficientNet'][['config','best_score','score']].iloc[-3:].values.tolist()
list_.reverse()
for l in list_:
    print("Score = %1.5f \n"% l[1], l[2],'\n',l[0])

Score = 0.92837 
 [0.9214176753998513, 0.9283730389397237, 0.9197130048595669, 0.9184037922331979, 0.9209934894547505] 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [0, 15],
 'gamma': 0.1,
 'learning_rate': 0.0001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0.001}

Score = 0.92472 
 [0.9235014785044339, 0.9162430155112288, 0.9247212963259768, 0.9074190399384591, 0.9240619948730036, 0.9226574977765508] 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [0, 15],
 'gamma': 0.3,
 'learning_rate': 0.0001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transfor

In [8]:
list_ = df[df['model']=='VGG16'][['config','best_score']].iloc[:3].values.tolist()
for l in list_:
    print("Score = %1.5f \n"% l[1],l[0])

df[df['model']=='VGG16'][['best_monotone','best_screenshot', 'best_unknown']].iloc[:3]
    

Score = 0.00000 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [5, 10],
 'gamma': 0.3,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [1, None],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0}

Score = 0.00000 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [0, 15],
 'gamma': 0.1,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0}

Score = 0.90091 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [0, 15],
 'gamma': 0.3,
 'learning_rate': 0.001,
 'loss_cls_weight'

,best_monotone,best_screenshot,best_unknown
20,0.000000,0.000000,0.00000
15,0.000000,0.000000,0.00000
17,0.950311,0.931559,0.87206


In [9]:
list_ = df[df['model']=='ResNet50'][['config','best_score']].iloc[:3].values.tolist()
for l in list_:
    print("Score = %1.5f \n"% l[1],l[0])

df[df['model']=='VGG16'][['best_monotone','best_screenshot', 'best_unknown']].iloc[:3]
    

Score = 0.89970 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [5, 10],
 'gamma': 0.1,
 'learning_rate': 0.0001,
 'loss_cls_weight': tensor([1.0000, 1.0000, 1.0000, 1.1000]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [1, None],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0.001}

Score = 0.89996 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [0, 15],
 'gamma': 0.3,
 'learning_rate': 1e-05,
 'loss_cls_weight': tensor([1.0000, 1.0000, 1.0000, 1.1000]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [1, None],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0}

Score = 0.90293 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [5, 10],
 'gamma': 0.1,
 'learning_rate': 1e-05,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.

,best_monotone,best_screenshot,best_unknown
20,0.000000,0.000000,0.00000
15,0.000000,0.000000,0.00000
17,0.950311,0.931559,0.87206


In [10]:
list_ = df[df['model']=='DenseNet121'][['config','best_score']].iloc[:3].values.tolist()
for l in list_:
    print("Score = %1.5f \n"% l[1],l[0])

df[df['model']=='VGG16'][['best_monotone','best_screenshot', 'best_unknown']].iloc[:3]
    

Score = 0.66395 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [5, 10],
 'gamma': 0.1,
 'learning_rate': 1e-05,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [1, None],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0.001}

Score = 0.87129 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [5, 10],
 'gamma': 0.1,
 'learning_rate': 1e-05,
 'loss_cls_weight': tensor([1.0000, 1.0000, 1.0000, 1.1000]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [1, None],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0}

Score = 0.89366 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [0, 15],
 'gamma': 0.3,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [1, None],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0.001}



,best_monotone,best_screenshot,best_unknown
20,0.000000,0.000000,0.00000
15,0.000000,0.000000,0.00000
17,0.950311,0.931559,0.87206


In [11]:
list_ = df[df['model']=='EfficientNet'][['config','best_score']].iloc[:3].values.tolist()
for l in list_:
    print("Score = %1.5f \n"% l[1],l[0])

df[df['model']=='VGG16'][['best_monotone','best_screenshot', 'best_unknown']].iloc[:3]
    

Score = 0.89628 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [0, 15],
 'gamma': 0.1,
 'learning_rate': 1e-05,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0}

Score = 0.89999 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [5, 10],
 'gamma': 0.1,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.2

,best_monotone,best_screenshot,best_unknown
20,0.000000,0.000000,0.00000
15,0.000000,0.000000,0.00000
17,0.950311,0.931559,0.87206


In [12]:
list_ = df[df['model']=='VGG16'][['config','best_score']].iloc[:3].values.tolist()
for l in list_:
    print("Score = %1.5f \n"% l[1],l[0])

df[df['model']=='VGG16'][['best_monotone','best_screenshot', 'best_unknown']].iloc[:3]
    

Score = 0.00000 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [5, 10],
 'gamma': 0.3,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [1, None],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0}

Score = 0.00000 
 {'batch_size': 128,
 'class_ratio': [0.5, 0.1, 0.15, 0.4],
 'epoch': [0, 15],
 'gamma': 0.1,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)],
 'val_ratio': [0.012, 0.25, 0.2, 0.2],
 'weight_decay': 0}

Score = 0.90091 
 {'batch_size': 128,
 'class_ratio': [0.6, 0.08, 0.1, 0.45],
 'epoch': [0, 15],
 'gamma': 0.3,
 'learning_rate': 0.001,
 'loss_cls_weight'

,best_monotone,best_screenshot,best_unknown
20,0.000000,0.000000,0.00000
15,0.000000,0.000000,0.00000
17,0.950311,0.931559,0.87206
